# Testing of features in polysaccharide2

In [ ]:
# Supressing annoying warnings (!must be done first!)
import warnings

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np

# Logging
from tqdm import tqdm as tqdm_text
from tqdm.notebook import tqdm as tqdm_notebook
from rich.progress import Progress, track

import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polysaccharide2 as ps2
from polysaccharide2.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

# Playing with ratios

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, ClassVar, TypeVar
from math import gcd
from numbers import Number


N = TypeVar('N')
def sgnmag(num : N) -> tuple[bool, N]:
    '''Returns the sign and magnitude of a numeric-like value'''
    return num < 0, abs(num)


@dataclass(repr=False)
class Ratio:
    '''For representing fractional ratios between two objects'''
    num   : Any
    denom : Any

    # REPRESENTATION
    def __repr__(self) -> str:
        return f'{self.num}/{self.denom}'
    
    def to_latex(self) -> str:
        '''Return latex-compatible string which represent fraction'''
        return rf'\frac{{{self.num}}}{{{self.denom}}}'

    # RELATIONS
    @property
    def reciprocal(self) -> 'Ratio':
        '''Return the reciprocal of a ration'''
        return self.__class__(self.denom, self.num)


@dataclass(repr=False)
class Rational(Ratio):
    '''For representing ratios of integers'''
    num   : int
    denom : int

    # REDUCTION
    autoreduce : ClassVar[bool]=False
    
    def __post_init__(self) -> None:
        if self.__class__.autoreduce:
            self.reduce()

    def reduce(self) -> None:
        '''Reduce numerator and denominator by greatest common factor'''
        _gcd = gcd(self.num, self.denom)
        self.num=int(self.num / _gcd)
        self.denom=int(self.denom / _gcd)
    simplify = reduce # alias for convenience

    @property
    def reduced(self) -> 'Rational':
        '''Return reduced Rational equivalent to the current rational (does not modify in-place)'''
        new_rat = self.__class__(self.num, self.denom)
        new_rat.reduce()

        return new_rat
    simplifed = reduced # alias for convenience
    
    def as_proper(self) -> tuple[int, 'Rational']:
        '''Returns the integer and proper fractional component of a ratio'''
        integ, remain = divmod(self.num, self.denom)
        return integ, self.__class__(remain, self.denom)
    
    # ARITHMETIC
    def __add__(self, other : 'Rational') -> 'Rational':
        '''Sum of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) + (self.denom * other.num),
            denom=(self.denom * other.denom)
        )
    
    def __sub__(self, other : 'Rational') -> 'Rational':
        '''Difference of two Rationals'''
        return self.__class__(
            num=(self.num * other.denom) - (self.denom * other.num),
            denom=(self.denom * other.denom)
        )

    def __mul__(self, other : 'Rational') -> 'Rational':
        '''Product of two Rationals'''
        return self.__class__(
            num=self.num * other.num,
            denom=self.denom * other.denom
        )

    def __div__(self, other : 'Rational') -> 'Rational':
        '''Quotient of two Rationals'''
        return self.__class__(
            num=self.num * other.denom,
            denom=self.denom * other.num
        )
    
    def __pow__(self, power : float) -> 'Rational':
        '''Exponentiates a ratio'''
        return self.__class__(
            num=self.num**power,
            denom=self.denom**power
        )

In [ ]:
p = Rational(3, 6)
q = Rational(4, 12)

print(p, p.reciprocal, p.reduced, p+q)

In [ ]:
Rational.autoreduce = False

In [ ]:
import numpy as np
from numbers import Number

for val in (4, 4.0, 4+0j, np.pi, '4', [4], False, 'sgdfg'):
    print(val, type(val), isinstance(val, Number))

In [ ]:
from fractions import Fraction

# Testing topology load and solvation

## Defining water

In [ ]:
from rdkit import Chem

from polysaccharide2.topology import offref
from polysaccharide2.topology.topIO import save_molecule
from polysaccharide2.rdutils.labeling.molwise import assign_ordered_atom_map_nums


water_dir = Path('water_files')
water_dir.mkdir(exist_ok=True)

# rdwat = Chem.MolFromSmiles('O')
# assign_ordered_atom_map_nums(rdwat, in_place=True)
# offwat = Molecule.from_rdkit(rdwat)
water = Molecule.from_smiles('O')

# offwat.to_file('wat.pdb', file_format='pdb')

TIP3P_ATOM_CHARGES = { # NOTE : units deliberately omitted here (become applied to entire charge array)
    'H' :  0.417,
    'O' : -0.843
}

water.partial_charges = [TIP3P_ATOM_CHARGES[atom.symbol] for atom in water.atoms]*offunit.elementary_charge

save_molecule(water_dir / 'water_tip3p_oe.sdf' , water, offref.TKREGS['OpenEye Toolkit'])
save_molecule(water_dir / 'water_tip3p_rd.sdf', water, offref.TKREGS['The RDKit'])
WATER_PATH = copyfile(water_dir / 'water_tip3p_oe.sdf', water_dir / 'water_tip3p.sdf')

### Method 1 : from .SDF file (must be curated via importlib_resources)

In [ ]:
sup = Chem.SDMolSupplier(str(water_dir / 'water_tip3p_rd.sdf'), sanitize=True, removeHs=False)
sup = Chem.SDMolSupplier(str(water_dir / 'water_tip3p_oe.sdf'), sanitize=True, removeHs=False)
water = next(sup)

offwat = Molecule.from_rdkit(water)
display(offwat)
print(offwat.partial_charges)
warnings.filterwarnings('ignore', category=DeprecationWarning) # doesn't actually seem to do anything about mbuild warnings

### Method 2 : from string block (can be included in .py file)

In [ ]:
from io import BytesIO

WATER_BLOCK_RD = '''\

    RDKit          2D

  3  2  0  0  0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    1.2990    0.7500    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.2990    0.7500    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0
  1  3  1  0
M  END
>  <atom.dprop.PartialCharge>  (1) 
-0.83399999999999996 0.41699999999999998 0.41699999999999998 

$$$$

'''

WATER_BLOCK_OE = '''
  -OEChem-09192311062D

  3  2  0     0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
    0.7500    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.3750   -0.6495    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0  0  0  0
  1  3  1  0  0  0  0
M  END
> <atom.dprop.PartialCharge>
-0.843000 0.417000 0.417000

$$$$
'''

with BytesIO(WATER_BLOCK_RD.encode('utf8')) as block_bytes:
	sup = Chem.ForwardSDMolSupplier(block_bytes, sanitize=True, removeHs=False)
	water2 = next(sup)

offwat2 = Molecule.from_rdkit(water2)
display(offwat2)
print(offwat2.partial_charges)

## Testing load using from_pdb

In [ ]:
from pathlib import Path

from polysaccharide2.topology import offref, topIO
from polysaccharide2.topology.topinfo import get_largest_offmol
from polysaccharide2.residues.partition import partition
from polysaccharide2.monomers.repr import MonomerGroup

pdb_dir  = Path('polymer_examples/compatible_pdbs')
mono_dir = Path('polymer_examples/monomer_generation/json_files/')

pdb_sub = 'simple_polymers'
# mol_name = 'PEO_PLGA'
# mol_name = 'paam_modified'
mol_name = 'polyvinylchloride'
# pdb_sub = 'proteins'
# mol_name = '6cww'

pdb = pdb_dir / pdb_sub / f'{mol_name}.pdb'
mono = mono_dir / f'{mol_name}.json'
assert(pdb.exists())
assert(mono.exists())

monogrp = MonomerGroup.from_file(mono)
rdmol = Chem.MolFromPDBFile(str(pdb))
offtop = Topology.from_pdb(pdb, _custom_substructures=monogrp.monomers, toolkit_registry=offref.TKREGS['The RDKit'])
was_partitioned = partition(offtop)
print(was_partitioned)

offmol = get_largest_offmol(offtop)

In [ ]:
from polysaccharide2.topology.offref import TKREGS
from polysaccharide2.topology import topIO
from polysaccharide2.residues.charging import application, calculation

offmol.assign_partial_charges(partial_charge_method='am1bccelf10', toolkit_registry=TKREGS['OpenEye Toolkit'])
res_chg = calculation.get_averaged_charges(offmol, monogrp)
offmol2 = application.apply_residue_charges(offmol, res_chg, in_place=False)

topIO.topology_to_sdf('pvc1.sdf', offmol.to_topology())
topIO.topology_to_sdf('pvc2.sdf', offmol2.to_topology())

## Solvation of Topologies

In [ ]:
from math import ceil
import numpy as np
import numpy.typing as npt

from polysaccharide2.topology.solvation import boxvectors, physprops
from openmm.unit import gram, centimeter, nanometer, mole, AVOGADRO_CONSTANT_NA
from openff.units.openmm import to_openmm as units_to_openmm


# PARAMETERS
density = 0.997 * (gram / centimeter**3)
exclusion = 1.3 * nanometer

# Sizing box vectors
water = Molecule.from_file(WATER_PATH)
mol_bbox = boxvectors.get_topology_bbox(offtop)
box_vecs = boxvectors.pad_box_vectors_uniform(mol_bbox, exclusion)
box_vol  = boxvectors.get_box_volume(box_vecs, units_as_openm=True)

# determining number of waters to place
N = physprops.num_mols_in_box(water.to_rdkit(), box_vol, density=density)
print(box_vol, N)

In [ ]:
topIO.topology_to_sdf('pvc.sdf', offtop)

In [ ]:
# PACKING
packtop = packmol.pack_box(
    [water],
    [N],
    offtop,
    # mass_density=1*offunit.gram/offunit.millilitre,
    box_vectors=box_vecs, 
    box_shape=packmol.UNIT_CUBE,
    center_solute='BRICK'
)

# Experimenting with SMARTS functional groups

In [ ]:
from polysaccharide2.monomers.substruct.functgroups import FN_GROUP_TABLE, FN_GROUP_ENTRIES
from polysaccharide2.monomers.substruct.functgroups.records import FnGroupSMARTSEntry

In [ ]:
FN_GROUP_TABLE.loc[FN_GROUP_TABLE['group_type'].str.contains('carbonyl')]

In [ ]:
smarts = FN_GROUP_ENTRIES[44].SMARTS
Chem.MolFromSmarts(smarts)

# Testing monomer loading

In [ ]:
from pathlib import Path 
from polysaccharide2.monomers.repr import MonomerGroup

p = Path('polymer_examples/monomer_generation/json_files/bisphenolA.json')
q = Path('polymer_examples/monomer_generation/json_files/naturalrubber.json')

mg1 = MonomerGroup.from_file(p)
mg2 = MonomerGroup.from_file(q)

In [ ]:
Chem.MolFromSmiles(mg2.monomers['naturalrubber'][0])

# Testing building

In [ ]:
from polysaccharide2.polymers import estimation, building

estimation.estimate_chain_len_linear(mg1, 10)

# Testing simulation I/O

In [ ]:
from pathlib import Path 
from openmm.unit import nanosecond

sp = ps2.openmmtools.records.SimulationParameters(100*nanosecond, 5, 'NVT')
sp.to_file(Path('test.json'))